In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sys,os
import string
import random
path = os.getcwd()

In [5]:
#utility function
def randomString(length) :
    l = string.ascii_lowercase
    return ''.join(random.choice(l) for i in range(length))

In [20]:
# rps : readingsPerSample
def exportFigs(input_filenames=["mHealth_subject3.log"],rps=50,figsize=(5,5),output_folder_prefex="",prefex="fig", activities = range(1,13)):
    counter = 0
    for input_filename in input_filenames:
        os.chdir(path)
        counter +=1
        df = pd.read_csv(input_filename, header=None, delim_whitespace=True)
        df = df[df[23]!=0] #drop 0 labeled features
        features_df = df[3]
        targets_df = df[23]

        #normalization
        features_np = features_df.values
        features_np_normalized = (features_np-min(features_np))/(max(features_np)-min(features_np))
        features_df_normalized = pd.DataFrame(features_np_normalized)

        #segmentation
        targets_dummy  = np.array([])
        features_dummy  = np.array([])
        for i in range(int(features_df_normalized.shape[0]/rps)):
            if  targets_df[i*rps:i*rps+rps].mean() in activities:
                features_dummy=np.append(features_dummy,features_df_normalized[i*rps:i*rps+rps])
                targets_dummy=np.append(targets_dummy,targets_df[i*rps:i*rps+rps].mean()-1)

        features_dummy=features_dummy.reshape(-1,rps)

        
        #folders hiararchy
        if output_folder_prefex == "":
            output_folder_prefex="Out"
        os.chdir(path)
        outputFolderName = output_folder_prefex+"_"+str(counter)
        try :
            os.makedirs(outputFolderName)
        except:
            pass
        path_str = os.path.join(str(path),outputFolderName)
        os.chdir(path_str)

        tmp=1
        imageCounter=1
        for i in range(0,len(targets_dummy)) :
            if int(targets_dummy[i]+1) != tmp:
                imageCounter = 0
                tmp=int(targets_dummy[i]+1)
            imageCounter+=1
            plt.figure(figsize=figsize)
            plt.xticks([])
            plt.yticks([])
            plt.rc("figure", facecolor="white")
            for spine in plt.gca().spines.values():
                spine.set_visible(False)
            plt.plot(features_dummy[i], color='k')
            label=str(int(targets_dummy[i]+1))
            print(label+"_fig_"+str(imageCounter)+".png")
            plt.savefig(label+"_fig_"+str(imageCounter)+".png")
            plt.clf()
            plt.cla()
            plt.close()
        os.chdir(path)

In [ ]:
exportFigs(input_filenames=["mHealth_subject3.log","mHealth_subject2.log"],figsize=(3,3),rps=50)